[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/car_dd_dataset_workshop/blob/main/06_using_3d_models.ipynb)


Note: If using in Google Colab, make sure you [install all the requirements listed here](https://github.com/harpreetsahota204/car_dd_dataset_workshop/blob/main/requirements.txt).

Which you can do by running the following:

```
# Install requirements directly from the URL
!pip install -r https://raw.githubusercontent.com/harpreetsahota204/car_dd_dataset_workshop/refs/heads/main/requirements.txt
```

You'll also want to connect to a GPU runtime.



# Using VGGT as a FiftyOne Remote Source Zoo Model


In [ ]:
import fiftyone as fo

# dataset = fo.load_dataset("cardd_from_hub")

# # or if you are in a new notebook
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "harpreetsahota/cardd_workshop_post_03",
    overwrite=True
    )

dataset.persistent=True

<div class="alert alert-warning">
<strong>Note:</strong> This notebook will be faster if you have access to GPU resources. The VGGT model is *relatively* lightweight (you can run it on CPU). But, it may take a while. 

If you want, you can download a dataset with all the generated 3D assets from the VGGT model, plus the enrichments from the previous notebook via the following:
</div>

You'll note we have to use a slightly different pattern for downloading 3D assets from Hugging Face. Let's just say we're working on making sure paths are working properly in FO3D.

In [ ]:
import json
import os

from huggingface_hub import snapshot_download

# Download the dataset snapshot to the current working directory

snapshot_download(repo_id="harpreetsahota/cardd_workshop_post_threed", local_dir="./cardd_threed", repo_type="dataset")

import fiftyone as fo

import fiftyone as fo
import os
import json

# Load dataset from current directory using FiftyOne's native format
dataset = fo.Dataset.from_dir(
    dataset_dir="./cardd_threed",  # Current directory contains the dataset files
    dataset_type=fo.types.FiftyOneDataset,  # Specify FiftyOne dataset format
    name="cardd_threed"  # Assign a name to the dataset for identification
)

def update_dataset_ply_paths(dataset):
    """
    Update PLY file paths in FiftyOne 3D dataset to use absolute paths.
    
    This function iterates through all samples in a FiftyOne dataset and modifies
    any PLY mesh file paths found in the JSON metadata to be absolute paths
    relative to the sample's directory location.
    
    Args:
        dataset (fiftyone.Dataset): A FiftyOne dataset containing 3D samples
                                   with JSON metadata files that may reference
                                   PLY mesh files with relative paths.
    
    Returns:
        None: The function modifies the JSON files in-place on disk.
    
    Note:
        - This function assumes each sample's filepath points to a JSON file
        - The JSON structure should contain a 'children' array with objects
        - PLY mesh objects are identified by '_type': 'PlyMesh'
        - Only objects with a 'plyPath' field will be updated
    """
    # Iterate through each sample in the dataset
    for sample in dataset:
        # Get the file path of the current sample (JSON metadata file)
        fo3d_filepath = sample.filepath
        
        # Extract the directory containing the sample file
        fo3d_directory = os.path.dirname(fo3d_filepath)
        
        # Read and parse the JSON metadata file
        with open(fo3d_filepath, 'r') as f:
            fo3d_data = json.load(f)
        
        # Process each child object in the JSON structure
        for child in fo3d_data.get('children', []):
            # Check if this child is a PLY mesh object with a path reference
            if child.get('_type') == 'PlyMesh' and 'plyPath' in child:
                # Convert relative PLY path to absolute path
                # by joining it with the sample's directory
                child['plyPath'] = os.path.join(fo3d_directory, child['plyPath'])
        
        # Write the updated JSON data back to the file
        with open(fo3d_filepath, 'w') as f:
            json.dump(fo3d_data, f, indent=2)  # Pretty-print with 2-space indentation

# Execute the path update function on the loaded dataset
update_dataset_ply_paths(dataset)

## Set up the model source

First, you need to register the model source. You can do so as shown here:

In [ ]:
import fiftyone.zoo as foz

foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/vggt",
    overwrite=True
)

Next, you need to instantiate the model.

Note, be sure to install the following:

- `vggt@git+https://github.com/facebookresearch/vggt.git`
- `open3d`

In [ ]:
model = foz.load_zoo_model(
    "facebook/VGGT-1B",
    mode="crop", # you can also pass "pad",
    confidence_threshold=0.7
    )

Finally, you can apply the model to your dataset:

In [ ]:
dataset.apply_model(model, "depth_map_path")

Note, we are saving only the paths to the depth map as a dummy field. We won't have these as a part of our original dataset, instead we will create a Grouped Dataset (shown below):

In [ ]:
import fiftyone as fo
import os
from pathlib import Path

# Get filepaths from your existing dataset
filepaths = dataset.values("filepath")

# Create a new grouped dataset
grouped_dataset = fo.Dataset("cardd_3d", overwrite=True)
grouped_dataset.add_group_field("group", default="rgb")

# Process each filepath and create the group structure
samples = []
for filepath in filepaths:
    # Extract base information from the filepath
    path = Path(filepath)
    base_dir = path.parent
    base_name = path.stem
    
    # Create paths for each modality following your pattern
    rgb_path = filepath  # Original filepath (RGB)
    depth_path = os.path.join(base_dir, f"{base_name}_depth.png")  # Depth map
    threed_path = os.path.join(base_dir, f"{base_name}.fo3d")  # 3D point cloud
    
    # Create a group for these related samples
    group = fo.Group()
    
    # Create a sample for each modality with the appropriate group element
    rgb_sample = fo.Sample(filepath=rgb_path, group=group.element("rgb"))
    depth_sample = fo.Sample(filepath=depth_path, group=group.element("depth"))
    threed_sample = fo.Sample(filepath=threed_path, group=group.element("threed"))
    
    # Add samples to the list
    samples.extend([rgb_sample, depth_sample, threed_sample])

# Add all samples to the dataset
grouped_dataset.add_samples(samples)
grouped_dataset.save()

Now we can view the results in the FiftyOne App

In [ ]:
fo.launch_app(grouped_dataset)